In [75]:
#%%
import sys
sys.path.append('/Users/linh.nguyen4')
import pandas as pd
import numpy as np
import os
import calendar
import psycopg2
from pathlib import Path
from loguru import logger
from dotenv import load_dotenv
from datetime import datetime, timedelta
from unidecode import unidecode
from avay_bq import AvayBQServiceAcc

from slack import WebClient
from slack.errors import SlackApiError

import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from math import floor, ceil

from avay_bq import AvayBQServiceAcc
from Downloads.da_utils.bigquery_service.BigqueryClient import BigqueryClient
from Downloads.da_utils.bigquery_service.BigqueryService import BigqueryService

from Downloads.da_utils.google_sheets_service.GoogleSheetsClient import GoogleSheetsClient
from Downloads.da_utils.google_sheets_service.GoogleSheetsService import GoogleSheetsService

from Downloads.da_utils.google_drive_service.GoogleDriveClient import GoogleDriveClient
from Downloads.da_utils.google_drive_service.GoogleDriveService import GoogleDriveService

from Downloads.da_utils.slack_service.SlackClient import SlackClient
from Downloads.da_utils.slack_service.SlackService import SlackService

load_dotenv()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 2000

In [76]:

drive_client = GoogleDriveClient(credentials_path=Path(os.environ.get("GOOGLE_CREDENTIALS_PATH")), drive_token_path=Path(os.environ.get("GOOGLE_DRIVE_TOKEN_PATH")))
drive_repo = GoogleDriveService(drive_client)

ggsheet_client = GoogleSheetsClient(credentials_path=Path(os.environ.get("GOOGLE_CREDENTIALS_PATH")), sheets_token_path=Path(os.environ.get("GG_SHEETS_TOKEN_PATH")))
ggsheet_repo = GoogleSheetsService(ggsheet_client)

slack_client = SlackClient(token=os.environ.get("SLACK_BOT_AUTHEN"))
slack_repo = SlackService(client=slack_client)

avay_bq_acc = AvayBQServiceAcc()

sa_ana = Path('/Users/linh.nguyen4/.config/ts-dp-v2.json')
adc = Path('/Users/linh.nguyen4/.config/gcloud/application_default_credentials.json')
bq_client = BigqueryClient(billing_project_id='prj-ts-p-analytic-8057',  adc_file_path= adc)
bq_repo = BigqueryService(bq_client)

bot_auth_token = os.environ.get("SLACK_BOT_AUTHEN")
user_auth_token = os.environ.get("SLACK_USER_AUTHEN")
clientSlack = WebClient(token = bot_auth_token)

channel_id = "C04MYU3L7LN"
user_id = "U0480LX5468"
linh_id = "U047QMYB9TQ"


In [77]:
def dayrange_recon(day_from_curr: int = 25) -> tuple:
    curr_time = datetime.now() 
    recon_time =(curr_time - timedelta(day_from_curr))
    _, end_date = calendar.monthrange(int(recon_time.strftime("%Y")), int(recon_time.strftime("%m")))

    recon_month = recon_time.strftime("%Y-%m")
    start_date = recon_time.strftime("%Y-%m") + "-" + "01"
    end_date = recon_time.strftime("%Y-%m") + "-" + str(end_date)
    return recon_month, start_date, end_date

In [78]:
recon_month, start_date, end_date = dayrange_recon()
recon_month

'2023-08'

In [79]:
# create directory
path = '/Users/linh.nguyen4/Downloads/04.Reconcile/06.TPB/nonsms_issued_card/'

import os

if not os.path.exists(f'{path}{recon_month}'):
    os.makedirs(f'{path}{recon_month}')

# GET CARD DATA

In [80]:
query = f""" 
WITH smv as (
  select 
  smv.*
  , DENSE_RANK() OVER (PARTITION BY lead_unique_token ORDER BY smv_issued_at ASC) AS rank
  from `prj-ts-p-lakehouse-8435.tpb.tpb_smv_in_dop_map` smv
) 

, tbl1 AS (
  SELECT distinct 
  application_id,
  CAST(JSON_VALUE(result_data, '$.id') AS INTEGER) AS offer_qualifications_id
  FROM `prj-ts-p-lakehouse-8435.dop.application_services`
)
, tbl2 AS (
  SELECT DISTINCT 
  id as offer_qualifications_id
  , CAST(JSON_VALUE(result, '$.other.lead_id') AS INTEGER) AS telco_lead_id
  FROM `prj-ts-p-lakehouse-8435.v1_proxy.offer_qualifications` 
)

, mapping AS (
  SELECT 
  tbl1.application_id
  , tbl2.telco_lead_id
  FROM tbl1 JOIN tbl2 USING(offer_qualifications_id)
)
    SELECT DISTINCT 
        dop.lead_unique_token
        , dop.application_id
        , app.lead_phone_number
        , app.lead_product_code
        , app.lead_telco_code
        , mapping.telco_lead_id 
        , cif
        , card_no
        , ten_kh
        , app.lead_created_at
        , smv_issued_at
        , smv_active_at
        , FORMAT_DATE('%Y-%m', DATE(smv.smv_issued_at)) as issue_month
        , FORMAT_DATE('%Y-%m', DATE(smv.smv_active_at)) as active_month
        , dop.mkt_channel
        , dop.utm_campaign
        , dop.mkt_channel_group AS channel
        , smv.smv_card_category as card_name
        , CASE  WHEN smv.smv_card_category LIKE '%SILVER%' THEN 'SILVER'
                WHEN smv.smv_card_category LIKE '%GOLD%' THEN 'GOLD'
                END AS card_class
        , dop.has_card_issued
        , dop.has_card_active
    FROM `prj-ts-p-analytic-8057.shared_tpb.tpb_dop_mapping` dop
    JOIN (select * from smv where rank=1) smv
        ON dop.lead_unique_token = smv.lead_unique_token
    JOIN `prj-ts-p-lakehouse-8435.dop.applications` app 
        ON dop.application_id = app.id
    LEFT JOIN mapping USING (application_id)
    
    WHERE 1=1
        AND has_card_issued = 1
        AND (dop.utm_campaign != 'referral01' OR dop.utm_campaign is null)
"""

df = bq_repo.get_data_from_query_into_pandas(query_str=query)
df.head()

,lead_unique_token,application_id,lead_phone_number,lead_product_code,lead_telco_code,telco_lead_id,cif,card_no,ten_kh,lead_created_at,smv_issued_at,smv_active_at,issue_month,active_month,mkt_channel,utm_campaign,channel,card_name,card_class,has_card_issued,has_card_active
0,UiK0gJpV,2360530,AYLZtHzw8GdUtknd4EZFbnD8iA+S+moaB+nWjdU=,tpb_02,viettel,<NA>,03755680,498796xxxxxx5116,NGUYEN LUONG DUC,2022-04-28 10:52:21+00:00,2022-04-28 13:23:52.000022,2022-05-06,2022-04,2022-05,MASAN ORGANIC,None,OTHER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1
1,eRFewoyZ,5788051,ARVK9KhMA6W3qy4XxoiP7UJdc4fWzLxiJ+7YTRc=,tpb_01,viettel,<NA>,10498844,498796xxxxxx5469,TRAN PHAN DUONG,2023-05-16 13:50:42+00:00,2023-05-16 15:07:40.999987,2023-05-16,2023-05,2023-05,AFFHY,None,PERFORMANCE MARKETING,VISA EVO SILVER CARD (9023),SILVER,1,1
2,aRFaORsV_1,2331057,AYaUUWbPgwVuZM+Uadw6G5pO4Tdh7Bm2y5v6oow=,tpb_02,mobifone,<NA>,06194478,498796xxxxxx6237,VU THI NGOC BICH,2022-04-27 12:30:49+00:00,2022-04-29 09:45:42.999984,2022-05-09,2022-04,2022-05,MASAN PREAPPROVAL,None,OTHER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1
3,YRFaPL3E_1,2345948,AegbPzWcK3hZ8q2kAHt+in6wyZyB56IklEJNMb8=,tpb_02,viettel,<NA>,06194615,498796xxxxxx9431,LE VIET DUNG,2022-04-27 12:36:22+00:00,2022-04-29 09:45:02.999980,NaT,2022-04,None,MASAN PREAPPROVAL,None,OTHER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,0
4,eRFfD1KU,6416978,Aflo9lMdGAnJBXnt6Li/foywdGuPQrJNDAHJlZc=,tpb_01,viettel,<NA>,04562375,498796xxxxxx2841,NGUYEN TRUNG ANH,2023-08-13 13:12:24+00:00,2023-08-13 14:24:46.999958,NaT,2023-08,None,VINAMILK,online_vnmilk_mass_200423,EVO PARTNER,VISA EVO SILVER CARD (9023),SILVER,1,0


In [81]:
def leadgen_pricing(df, recon_month):
    df['lead_source'] = np.select([df['mkt_channel'].str.contains('WITH OFFER'), 
                                         df['mkt_channel'].str.contains('WITHOUT OFFER'),
                                         ~df['mkt_channel'].isna()],
                                        ['SMS WITH OFFER', 
                                         'SMS WITHOUT OFFER', 
                                         'EVO/DIGITAL'])
    
    df['lead_cost'] = 900000

    # df['lead_cost'] = np.select([df['card_class'] == 'SILVER', 
    #                                    df['card_class'] == 'GOLD'], 
    #                                   [600000,
    #                                    1200000],
    #                                   default=np.nan)

    df['lead_created_at'] = pd.to_datetime(df['lead_created_at'], format='%Y-%m-%d').dt.tz_localize(None)

    df = df.query("(card_class in ['GOLD', 'SILVER']) & (issue_month == @recon_month)")  

    return df

tpb_card = leadgen_pricing(df, recon_month=recon_month)

In [82]:
tpb_card[['channel','lead_source']].drop_duplicates()

,channel,lead_source
4,EVO PARTNER,EVO/DIGITAL
67,PERFORMANCE MARKETING,EVO/DIGITAL
600,SMS WITHOUT OFFER,SMS WITHOUT OFFER
2305,OTHER,EVO/DIGITAL
6651,EVO BRANDING,EVO/DIGITAL
96303,SMS WITH OFFER,SMS WITH OFFER


In [83]:
tpb_card[['card_class','lead_cost']].drop_duplicates()

,card_class,lead_cost
4,SILVER,900000
67,GOLD,900000


In [84]:
tpb_card[['issue_month']].drop_duplicates()

,issue_month
4,2023-08


In [85]:
df1 = tpb_card.groupby(['issue_month', 'lead_source', 'card_class']).agg(
    issue_cnt=('has_card_issued', 'sum'),
    active_cnt=('has_card_active', 'sum'),
    lead_cost=('lead_cost', 'sum')
).reset_index()

df1 = df1[df1['lead_source'].notna()].sort_values(['issue_month', 'lead_source', 'card_class'], ascending=False)

df1 

,issue_month,lead_source,card_class,issue_cnt,active_cnt,lead_cost
5,2023-08,SMS WITHOUT OFFER,SILVER,41,36,36900000
4,2023-08,SMS WITHOUT OFFER,GOLD,30,27,27000000
3,2023-08,SMS WITH OFFER,SILVER,9,9,8100000
2,2023-08,SMS WITH OFFER,GOLD,12,10,10800000
1,2023-08,EVO/DIGITAL,SILVER,1357,1195,1221300000
0,2023-08,EVO/DIGITAL,GOLD,962,870,865800000


# UPDATE GDRIVE

In [86]:
#xlsx
tpb_card.to_excel(f"{path}{recon_month}/tpb_leadgen_recon_{recon_month}.xlsx", index=False)

In [87]:
def create_or_get_folder_sheet(recon_month, df, sheet_name, parent_folder_id):
    # Check if folder exists
    folder_check = drive_repo.list_folder_names(parent_folder_id =  parent_folder_id)
    if recon_month not in folder_check:
        folder_id = drive_repo.create_folder(folder_name= recon_month, parent_folder_id =  parent_folder_id)
    else:
        folder_id = drive_repo.get_folders(folder_name= recon_month, parent_folder_id =  parent_folder_id)
    
    # Check if sheet exists
    sheet_check = drive_repo.list_file_names(parent_folder_id= folder_id[0])
    
    if sheet_name not in sheet_check:
        wb = ggsheet_repo.create_spreadsheet(spreadsheet_name = sheet_name,parent_folder_id = folder_id[0])
        wb.add_worksheet(title="Details", rows=100, cols=20)
        wb.del_worksheet(wb.worksheet('Sheet1'))
        ws = wb.worksheet('Details')
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = df, starting_cell='A1')
    else:
        spreadsheet_key = drive_repo.get_files(file_name = sheet_name,parent_folder_id = folder_id[0])
        wb = ggsheet_repo.open_spreadsheet(spreadsheet_key = spreadsheet_key[0])
        ws = wb.worksheet('Details')
        ws.clear()
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = df, starting_cell='A1')
    return wb


In [88]:
parent_folder_id = '11QuXavTTWfBQXAB8m-n9JFBTavdY5uEU'
TPB_leadgen_issued_wb = create_or_get_folder_sheet(recon_month=recon_month, df=tpb_card, sheet_name="TPB_leadgen_issued_" + recon_month, parent_folder_id=parent_folder_id)

In [89]:
def create_or_update_sheet(wb, new_sheet, input_df):
    sheet_names = [worksheet.title for worksheet in wb.worksheets()]
    if new_sheet not in sheet_names:
        wb.add_worksheet(title=new_sheet, rows=100, cols=20)
        ws = wb.worksheet(new_sheet)
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell='A1')
    else:
        ws = wb.worksheet(new_sheet)
        ws.clear()
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell='A1')
    return

In [90]:
summary_data = tpb_card.groupby(['issue_month', 'lead_source', 'card_class']).agg({'has_card_issued':'sum', 'lead_cost':'sum'}).reset_index()
Summary_by_channel = tpb_card.groupby(['issue_month', 'lead_source', 'channel', 'card_class']).agg({'has_card_issued':'sum', 'lead_cost':'sum'}).reset_index()

create_or_update_sheet(TPB_leadgen_issued_wb, "Summary", summary_data)
create_or_update_sheet(TPB_leadgen_issued_wb, "Summary_by_channel (internal)", Summary_by_channel)

# UPDATE DISBURSAL

In [91]:
tpb_sms = tpb_card[tpb_card['lead_source'] == 'SMS WITH OFFER']
tpb_sms.head()


,lead_unique_token,application_id,lead_phone_number,lead_product_code,lead_telco_code,telco_lead_id,cif,card_no,ten_kh,lead_created_at,smv_issued_at,smv_active_at,issue_month,active_month,mkt_channel,utm_campaign,channel,card_name,card_class,has_card_issued,has_card_active,lead_source,lead_cost
96303,UiK94yLH,6173228,AV6Q4phan/UjUw8HzDuqO1H/Ik4dPFkotNzIjGs=,tpb_01,mobifone,3505712,11712634,498796xxxxxx3737,LUU NGOC QUYNH ANH,2023-07-14 16:25:52,2023-08-13 16:52:18.999984,NaT,2023-08,None,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,0,SMS WITH OFFER,900000
96727,kRXv1qo-,6270292,AS9TGItbMYM6N4Y6bLNxp8cKz9mfVR7BqoZohHQ=,tpb_01,mobifone,3519557,03221257,498796xxxxxx0941,LE THI THANH TUYEN,2023-07-27 11:59:40,2023-08-02 00:36:00.000000,2023-08-09,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA EVO SILVER CARD (9023),SILVER,1,1,SMS WITH OFFER,900000
97633,aRFfA7LL,6321753,AWV43b9te7jyiKF5obK1gmdLxFVoksXCWmZnYDc=,tpb_01,mobifone,3520628,01782086,498796xxxxxx8988,LUONG AN PHO,2023-08-02 16:39:12,2023-08-02 17:57:19.999987,2023-08-02,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA EVO SILVER CARD (9023),SILVER,1,1,SMS WITH OFFER,900000
98984,QiK972ZN,6223460,AQ5Cq+KFalLD2LyLWz9f81/0PsqYiLf8UFjFo5Y=,tpb_01,mobifone,3516917,00624267,498796xxxxxx2961,NGUYEN DANG PHU,2023-07-21 09:46:30,2023-08-11 19:41:35.000044,2023-08-11,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA EVO SILVER CARD (9023),SILVER,1,1,SMS WITH OFFER,900000
99007,YRFfAwNu,6316141,Aes2QNkwisrMSVy0+5qrQ8LZHlb18foqaDWax7s=,tpb_01,mobifone,3520607,11814133,498796xxxxxx6464,PHAM NGOC LOAN,2023-08-01 23:35:01,2023-08-02 01:34:43.000003,2023-08-04,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1,SMS WITH OFFER,900000


In [92]:
tpb_sms.count()

lead_unique_token    21
application_id       21
lead_phone_number    21
lead_product_code    21
lead_telco_code      21
telco_lead_id        21
cif                  21
card_no              21
ten_kh               21
lead_created_at      21
smv_issued_at        21
smv_active_at        14
issue_month          21
active_month         14
mkt_channel          21
utm_campaign          0
channel              21
card_name            21
card_class           21
has_card_issued      21
has_card_active      21
lead_source          21
lead_cost            21
dtype: int64

## get ts_lead_id

In [93]:
v1_query = f"""
select distinct
CAST(id as INT64) as lead_id
, phone_number
, CAST(telco_lead_id as INT64) as telco_lead_id
, date(created_at) as lead_created_at
from `prj-ts-p-lakehouse-8435.v1_proxy.phone_infos`
where true
and bank_code = 'tpbank'
                        ;
"""
v1_df = bq_repo.get_data_from_query_into_pandas(query_str= v1_query)

In [94]:
def generate_tpb_sms_map(tpb_sms, v1_df):
    def to_categorical(tpb_sms_map):
        return pd.merge(tpb_sms_map, 
                         v1_df[['phone_number', 'telco_lead_id', 'lead_id']],
                         how='left',
                         left_on=['telco_lead_id', 'lead_phone_number'],
                         right_on=['telco_lead_id', 'phone_number']).dropna(subset=['telco_lead_id']).drop_duplicates()

    tpb_sms_map = to_categorical(tpb_sms)

    def to_categorical_na(tpb_sms):
        tpb_sms_na = tpb_sms[tpb_sms['telco_lead_id'].isna()].drop(columns=['telco_lead_id'])
        return pd.merge(tpb_sms_na, 
                         v1_df[['phone_number', 'telco_lead_id', 'lead_id']],
                         how='left',
                         left_on='lead_phone_number',
                         right_on='phone_number').drop_duplicates()

    tpb_sms_map_na = to_categorical_na(tpb_sms)

    def combine_tpb_sms_maps(tpb_sms_map, tpb_sms_map_na):
        tpb_sms_map_full = pd.concat([tpb_sms_map, tpb_sms_map_na])
        return tpb_sms_map_full.drop_duplicates()

    tpb_sms_map_full = combine_tpb_sms_maps(tpb_sms_map, tpb_sms_map_na)

    return tpb_sms_map_full

tpb_sms_map = generate_tpb_sms_map(tpb_sms, v1_df)

display(tpb_sms_map.head())

,lead_unique_token,application_id,lead_phone_number,lead_product_code,lead_telco_code,telco_lead_id,cif,card_no,ten_kh,lead_created_at,smv_issued_at,smv_active_at,issue_month,active_month,mkt_channel,utm_campaign,channel,card_name,card_class,has_card_issued,has_card_active,lead_source,lead_cost,phone_number,lead_id
0,UiK94yLH,6173228,AV6Q4phan/UjUw8HzDuqO1H/Ik4dPFkotNzIjGs=,tpb_01,mobifone,3505712,11712634,498796xxxxxx3737,LUU NGOC QUYNH ANH,2023-07-14 16:25:52,2023-08-13 16:52:18.999984,NaT,2023-08,None,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,0,SMS WITH OFFER,900000,AV6Q4phan/UjUw8HzDuqO1H/Ik4dPFkotNzIjGs=,14218490
1,kRXv1qo-,6270292,AS9TGItbMYM6N4Y6bLNxp8cKz9mfVR7BqoZohHQ=,tpb_01,mobifone,3519557,03221257,498796xxxxxx0941,LE THI THANH TUYEN,2023-07-27 11:59:40,2023-08-02 00:36:00.000000,2023-08-09,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA EVO SILVER CARD (9023),SILVER,1,1,SMS WITH OFFER,900000,AS9TGItbMYM6N4Y6bLNxp8cKz9mfVR7BqoZohHQ=,14223519
2,aRFfA7LL,6321753,AWV43b9te7jyiKF5obK1gmdLxFVoksXCWmZnYDc=,tpb_01,mobifone,3520628,01782086,498796xxxxxx8988,LUONG AN PHO,2023-08-02 16:39:12,2023-08-02 17:57:19.999987,2023-08-02,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA EVO SILVER CARD (9023),SILVER,1,1,SMS WITH OFFER,900000,AWV43b9te7jyiKF5obK1gmdLxFVoksXCWmZnYDc=,14224086
3,QiK972ZN,6223460,AQ5Cq+KFalLD2LyLWz9f81/0PsqYiLf8UFjFo5Y=,tpb_01,mobifone,3516917,00624267,498796xxxxxx2961,NGUYEN DANG PHU,2023-07-21 09:46:30,2023-08-11 19:41:35.000044,2023-08-11,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA EVO SILVER CARD (9023),SILVER,1,1,SMS WITH OFFER,900000,AQ5Cq+KFalLD2LyLWz9f81/0PsqYiLf8UFjFo5Y=,14222433
4,YRFfAwNu,6316141,Aes2QNkwisrMSVy0+5qrQ8LZHlb18foqaDWax7s=,tpb_01,mobifone,3520607,11814133,498796xxxxxx6464,PHAM NGOC LOAN,2023-08-01 23:35:01,2023-08-02 01:34:43.000003,2023-08-04,2023-08,2023-08,SMS-MBF WITH OFFER,None,SMS WITH OFFER,VISA CREDIT GOLD EVO CARD (9024),GOLD,1,1,SMS WITH OFFER,900000,Aes2QNkwisrMSVy0+5qrQ8LZHlb18foqaDWax7s=,14224060


In [95]:
tpb_sms_map.count()

lead_unique_token    21
application_id       21
lead_phone_number    21
lead_product_code    21
lead_telco_code      21
telco_lead_id        21
cif                  21
card_no              21
ten_kh               21
lead_created_at      21
smv_issued_at        21
smv_active_at        14
issue_month          21
active_month         14
mkt_channel          21
utm_campaign          0
channel              21
card_name            21
card_class           21
has_card_issued      21
has_card_active      21
lead_source          21
lead_cost            21
phone_number         21
lead_id              21
dtype: int64

In [96]:
def preprocess_disbursed(input_df):
    output_df = input_df.copy()
    output_df['bank_code'] = 'tpbank'
    output_df['telco_lead_id'] = output_df['telco_lead_id'].astype(int)
    output_df['lead_id'] = output_df['lead_id'].astype(int, errors='ignore')
    output_df['lead_source'] = output_df['lead_source'].astype(str)
    output_df['phone_number'] = output_df['lead_phone_number'].astype(str)
    output_df['nid'] = pd.NA
    output_df['lead_unique_token'] = output_df['lead_unique_token'].astype(str)
    output_df['lead_created_date'] = pd.to_datetime(output_df['lead_created_at']).dt.date
    output_df['channel'] = 'esms'
    output_df['product_code'] = output_df['lead_product_code'].astype(str)
    output_df['card_name'] = output_df['card_name'].astype(str)
    output_df['card_class'] = output_df['card_class'].astype(str)
    output_df['telco_code'] = output_df['lead_telco_code'].astype(str)
    output_df['issued_date'] = pd.to_datetime(output_df['smv_issued_at']).dt.date
    output_df['activated_date'] = pd.to_datetime(output_df['smv_active_at']).dt.date
    output_df['commission'] = output_df['lead_cost'].astype(float)
    output_df['other'] = pd.NA
    output_df = output_df[['bank_code', 'telco_lead_id', 'lead_id', 'lead_source', 'phone_number',
                           'nid', 'lead_unique_token', 'lead_created_date', 'channel', 'product_code',
                           'card_name', 'card_class', 'telco_code', 'issued_date', 'activated_date', 
                           'commission', 'other']]
    return output_df


In [97]:
disbursal_df = preprocess_disbursed(tpb_sms_map)

In [98]:
disbursal_df.head()

,bank_code,telco_lead_id,lead_id,lead_source,phone_number,nid,lead_unique_token,lead_created_date,channel,product_code,card_name,card_class,telco_code,issued_date,activated_date,commission,other
0,tpbank,3505712,14218490,SMS WITH OFFER,AV6Q4phan/UjUw8HzDuqO1H/Ik4dPFkotNzIjGs=,<NA>,UiK94yLH,2023-07-14,esms,tpb_01,VISA CREDIT GOLD EVO CARD (9024),GOLD,mobifone,2023-08-13,NaT,900000.0,<NA>
1,tpbank,3519557,14223519,SMS WITH OFFER,AS9TGItbMYM6N4Y6bLNxp8cKz9mfVR7BqoZohHQ=,<NA>,kRXv1qo-,2023-07-27,esms,tpb_01,VISA EVO SILVER CARD (9023),SILVER,mobifone,2023-08-02,2023-08-09,900000.0,<NA>
2,tpbank,3520628,14224086,SMS WITH OFFER,AWV43b9te7jyiKF5obK1gmdLxFVoksXCWmZnYDc=,<NA>,aRFfA7LL,2023-08-02,esms,tpb_01,VISA EVO SILVER CARD (9023),SILVER,mobifone,2023-08-02,2023-08-02,900000.0,<NA>
3,tpbank,3516917,14222433,SMS WITH OFFER,AQ5Cq+KFalLD2LyLWz9f81/0PsqYiLf8UFjFo5Y=,<NA>,QiK972ZN,2023-07-21,esms,tpb_01,VISA EVO SILVER CARD (9023),SILVER,mobifone,2023-08-11,2023-08-11,900000.0,<NA>
4,tpbank,3520607,14224060,SMS WITH OFFER,Aes2QNkwisrMSVy0+5qrQ8LZHlb18foqaDWax7s=,<NA>,YRFfAwNu,2023-08-01,esms,tpb_01,VISA CREDIT GOLD EVO CARD (9024),GOLD,mobifone,2023-08-02,2023-08-04,900000.0,<NA>


In [99]:
disbursal_df.count()

bank_code            21
telco_lead_id        21
lead_id              21
lead_source          21
phone_number         21
nid                   0
lead_unique_token    21
lead_created_date    21
channel              21
product_code         21
card_name            21
card_class           21
telco_code           21
issued_date          21
activated_date       14
commission           21
other                 0
dtype: int64

In [100]:
file_path = f"{path}{recon_month}/tpb_leadgen_recon_{recon_month}.csv"
disbursal_df.to_csv(file_path, index=False)

In [101]:
file_path: Path = Path(file_path)

# Send success message to channel
clientSlack.chat_postMessage(**{"text": f"<@{user_id}> update leadgen card TPB {recon_month}"}, channel=channel_id)

# Upload file to channel
filetype = file_path.suffix[1:]
filename = file_path.name
with open(file_path, "rb") as file:
	clientSlack.files_upload(channels=channel_id, file=file, title=filename, filetype=filetype)


/Users/linh.nguyen4/Library/Python/3.9/lib/python/site-packages/slack_sdk/web/internal_utils.py:450: UserWarning: client.files_upload() may cause some issues like timeouts for relatively large files. Our latest recommendation is to use client.files_upload_v2(), which is mostly compatible and much stabler, instead.
  warnings.warn(message)
